In [1]:
from glob import glob
import re
import ftfy

input_text = []
labels = []
fillers = [re.compile(r, re.IGNORECASE) for r in ["u+(h|m)+", "like"]]

def replace(match):
    return match.group(0).upper()

for file in glob("./ml-stuttering-events-dataset/*.orth"):
    data = ftfy.fix_text(open(file, "rb").read().decode(errors="replace"))
    data = re.sub(r"\{([A-Za-z0-9]|\s)+?\}\s?", "", data)
    for filler in fillers:
        data = filler.sub(replace, data)

    words = [a for a in data.split(" ") if a]
    is_filler = []
    for word in words:
        is_filler.append(word.upper() == word and word.lower() != "i")

    input_text.extend(word.lower() for word in words)
    labels.extend(is_filler)

input_text

['most',
 'of',
 'the',
 'programming',
 'i',
 'do',
 'is',
 'in',
 'visual',
 'basic',
 'um,',
 'which',
 'is',
 'in',
 'windows.',
 'um,',
 'at',
 'my',
 'school',
 "they've",
 'got',
 'visual',
 'basic,',
 'and',
 'i',
 'sometimes',
 'spend',
 'some',
 'time',
 'writing',
 'in',
 'there,',
 'in',
 'our',
 'i.t.',
 'lessons,',
 'information',
 'technology.',
 'um',
 'um',
 'there',
 'were',
 'few',
 'samples,',
 'um',
 'supplied',
 'from',
 'visual',
 'basic',
 'and,',
 'i',
 'picked',
 'up',
 'some',
 'the',
 'techniques',
 'from',
 'that,',
 'with',
 'all',
 'the',
 'add-ons',
 'and',
 'everything,',
 "it's",
 'really',
 'a',
 'very',
 'u',
 'user',
 'friendly',
 'language',
 'and',
 'easy',
 'to',
 'use.',
 'um,',
 'at',
 'home',
 "i've",
 'actually',
 'got',
 'a',
 'higher',
 'version',
 'of',
 'visual',
 'basic',
 'than',
 'at',
 'school,',
 'um',
 'but',
 'the',
 'computer',
 'sometimes',
 'crashes',
 'when',
 'it',
 'runs',
 'that,',
 'i',
 'think',
 "it's",
 'because',
 'of',

In [2]:
print(" ".join(input_text))

most of the programming i do is in visual basic um, which is in windows. um, at my school they've got visual basic, and i sometimes spend some time writing in there, in our i.t. lessons, information technology. um um there were few samples, um supplied from visual basic and, i picked up some the techniques from that, with all the add-ons and everything, it's really a very u user friendly language and easy to use. um, at home i've actually got a higher version of visual basic than at school, um but the computer sometimes crashes when it runs that, i think it's because of memory problems. um, yeah. and i've made some exe versions of some of my games, um but i'm keeping those to myself f for the time, i might sell them later when i'm older. um. what sort of games have you written? usually a strategy games, um, i've written a program where you�ve got to fill in some magic squares, and i've got a program to actually make a magic square any size you want. once i made one twentynine by twenty

In [3]:
import random
random.seed(42)

data = list(zip(input_text, labels))
chopped = []
last_idx = 0
def decision(probability):
    return random.random() < probability

while len(data) > 0:
    value = random.randrange(5, 20)
    chopped.append(data[:value])
    data = data[value:]

In [4]:
import datasets
tokens = [[a[0] for a in c] for c in chopped]
labels = [[1 if a[1] else 0 for a in c] for c in chopped]

ds = datasets.Dataset.from_dict({"labels": labels, "tokens": tokens})
ds.save_to_disk("stutter-ds")

Saving the dataset (0/1 shards):   0%|          | 0/606 [00:00<?, ? examples/s]